In [6]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
import pandas as pd
from tensorflow.keras.utils import to_categorical
import cv2
import time 
from skimage.transform import resize

    
    


model = tf.keras.models.load_model("animal_cnn_model.h5")
classes = ['bird', 'cat', 'deer', 'dog', 'rabbit', 'raccoon', 'squirrel']

output = []
boxes = []

#cap = cv2.VideoCapture(0) #webcam
cap=cv2.VideoCapture('deer.mp4') #easy video
#cap=cv2.VideoCapture('backyard.mp4') #hard video 
ret1,frame1= cap.read()
gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
gray1 = cv2.GaussianBlur(gray1, (19, 19), 1)

frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
out = cv2.VideoWriter('output.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width,frame_height))

 


#start_time = time.time()

while(cap.isOpened()):
   # elapsed_time = time.time() - start_time
#     if not motion:
#         ret1,frame1= cap.read()
#         gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
#         gray1 = cv2.GaussianBlur(gray1, (19, 19), 1)
        
#     motion = False
    
    
    ret2,frame2=cap.read()
    if not(ret2):
        break
    gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.GaussianBlur(gray2, (19, 19), 1)
    
    deltaframe=cv2.absdiff(gray1,gray2)
   # cv2.imshow("delta",deltaframe)
    threshold = cv2.threshold(deltaframe, 50, 255, cv2.THRESH_BINARY)[1]
    thresh_frame = cv2.dilate(threshold, None, iterations = 2) 
   # cv2.imshow("treshold",threshold)
    countour,heirarchy = cv2.findContours(threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for i in countour:
        if cv2.contourArea(i) < 3000:
            motion = True
            continue
      
 
        (x, y, w, h) = cv2.boundingRect(i)
    
#         test = str(x) + str(y) + str(w) + str(h)
#         if test not in boxes:
#             boxes.append(test)
        cv2.rectangle(frame2, (x, y), (x + w, y + h), (0, 0, 225), 2)
        obj = frame2[y:y+h,x:x+w]
        cv2.imwrite("test.jpg",obj)


        obj_re = resize(obj, (224,224,3))


        obj_re = obj_re.reshape((1,) + obj_re.shape)



        predicts = model.predict(obj_re)
        index = np.argsort(predicts[0,:])

        if predicts[0,index[6]] > .85:
            print("Most likely class:", classes[index[6]], "-- Probability:", predicts[0,index[6]])
            cv2.rectangle(frame2, (x, y), (x + w, y + h), (0, 225, 0), 2)

           
            temp = { "detected": {
                        "class": classes[index[6]],
                        "prob":predicts[0,index[6]],
                        "x": x,
                        "y": y,
                        "w": w,
                        "h": h

                        }}

            output.append(temp)


            frame2 = cv2.putText(frame2,classes[index[6]] , (50, 50), cv2.FONT_HERSHEY_PLAIN, 4,(255,0,0), 2);
    out.write(frame2)
    cv2.imshow('Tracker',frame2)
    
    
    if cv2.waitKey(30) == 27:
      break
out.release()
cap.release()
cv2.destroyAllWindows()



def find_preds(predicts):
    actual_pred =[]
    for p in predicts:
        max_i=0
        for i in range(1,len(p)):
            if p[i] > p[max_i]:
                max_i = i
        actual_pred.append(max_i)
    return actual_pred





Most likely class: cat -- Probability: 0.86516297
Most likely class: cat -- Probability: 0.8952174
Most likely class: cat -- Probability: 0.9315587
Most likely class: cat -- Probability: 0.97441566
Most likely class: cat -- Probability: 0.96193326
Most likely class: cat -- Probability: 0.9344679
Most likely class: squirrel -- Probability: 0.8603048
Most likely class: cat -- Probability: 0.89508176
Most likely class: deer -- Probability: 0.9613631
Most likely class: deer -- Probability: 0.9817556
Most likely class: deer -- Probability: 0.9896836
Most likely class: deer -- Probability: 0.9979115
Most likely class: deer -- Probability: 0.9925754
Most likely class: deer -- Probability: 0.94376796
Most likely class: deer -- Probability: 0.8656989
Most likely class: deer -- Probability: 0.86815864
Most likely class: deer -- Probability: 0.9695365
Most likely class: deer -- Probability: 0.9908695
Most likely class: deer -- Probability: 0.99016273
Most likely class: deer -- Probability: 0.9928

Most likely class: deer -- Probability: 0.9945622
Most likely class: deer -- Probability: 0.9935434
Most likely class: deer -- Probability: 0.99399745
Most likely class: deer -- Probability: 0.9945328
Most likely class: deer -- Probability: 0.9932935
Most likely class: deer -- Probability: 0.9955505
Most likely class: deer -- Probability: 0.9944834
Most likely class: deer -- Probability: 0.995961
Most likely class: deer -- Probability: 0.99514633
Most likely class: deer -- Probability: 0.99539775
Most likely class: deer -- Probability: 0.99650663
Most likely class: deer -- Probability: 0.989675
Most likely class: deer -- Probability: 0.92540324
Most likely class: cat -- Probability: 0.9518442
Most likely class: cat -- Probability: 0.90620095
Most likely class: cat -- Probability: 0.9570232
Most likely class: cat -- Probability: 0.96345675
Most likely class: cat -- Probability: 0.90980345
Most likely class: cat -- Probability: 0.9062888
